In [23]:
API_KEY = '6d4bd60e237eae6df24af2c0189a10278154d4f2079dec475b7d15ca11baea04'
#instantiate query api
from sec_api import QueryApi
queryApi = QueryApi(api_key=API_KEY) 

In [24]:
import sys
import subprocess
import pkg_resources
# import os 

print(sys.executable)
!{sys.executable} -m pip install openpyxl
def check_package(package_name):
    try:
        dist = pkg_resources.get_distribution(package_name)
        print(f'{package_name} ({dist.version}) is installed')
        return True
    except pkg_resources.DistributionNotFound:
        print(f'{package_name} is NOT installed')
        return False

print("Python executable:", sys.executable)
print("\nChecking openpyxl installation:")
check_package('openpyxl')



c:\Users\Dafa_1\venv\Scripts\python.exe
Python executable: c:\Users\Dafa_1\venv\Scripts\python.exe

Checking openpyxl installation:
openpyxl (3.1.5) is installed


True

In [25]:
import pandas as pd
from time import sleep 
import concurrent.futures
import time
file_path = "D:/coding/Nanik/SEC/backend/sec_app/data/stocks_perf_data.xlsx"
df = pd.read_excel(file_path)
cik_lookup = df.set_index('Symbol')['CIK Code'].to_dict()

tickers = df['Symbol'].dropna().unique().tolist()[:10]
print("Processing these 10 tickers:", tickers)

Processing these 10 tickers: ['A', 'AA', 'AACB', 'AACG', 'AACT', 'AADI', 'AAL', 'AAM', 'AAME', 'AAMI']


In [26]:
def get_filings(query):
  from_param = 0
  size_param = 10 
  all_filings = []

  while True:
    query['from'] = from_param
    query['size'] = size_param

    response = queryApi.get_filings(query)
    print(response)
    filings = response['filings']

    if len(filings) == 0:
      break

    all_filings.extend(filings)

    if len(all_filings) >= 10:
        all_filings = all_filings[:10]  # Ensure we only keep the first 10
        break

    from_param += size_param

  return pd.json_normalize(all_filings)

In [27]:
def fetch_filings_for_ticker(ticker):
  form_type_query  = 'formType:("10-K") AND NOT formType:("10-K/A", "10-Q/A", NT)'
  ticker_query     = f'ticker:({ticker})'
  date_range_query = 'filedAt:[2020-01-01 TO 2025-03-15]'
  lucene_query = f"{form_type_query} AND {ticker_query} AND {date_range_query}"

  search_query = {
    "query": lucene_query,
    "from": "0",
    "size": "200",
    "sort": [{ "filedAt": { "order": "desc" } }]
  }
 
  filings_data = []


  try:
        response = queryApi.get_filings(search_query)
        if response and 'filings' in response:
            filings = response['filings']
            for filing in filings:
                cik = str(cik_lookup.get(ticker, '')).zfill(10)
                link = filing.get('linkToFilingDetails', '')
                accession_number = ''
                if link:
                    parts = link.split('/')
                    if len(parts) > 7:
                        accession_number = parts[7].replace('-', '')
                filing_data = {
                    'ticker': ticker,
                    'formType': filing.get('formType', ''),
                    'periodOfReport': filing.get('periodOfReport', ''),
                    'filedAt': filing.get('filedAt', ''),
                    'linkToFilingDetails': link,
                    'cik': cik,
                    'accessionNumber': accession_number
                    }
                filings_data.append(filing_data)
  except Exception as e:
            print(f"Error processing {ticker}: {e}")
        
  return filings_data
print("Sample CIK mappings:")
for ticker, cik in list(cik_lookup.items())[:5]:
    print(f"{ticker}: {cik}")

all_filings = []
number_of_workers = 15

with concurrent.futures.ThreadPoolExecutor(max_workers=number_of_workers) as executor:
    future_to_ticker = {executor.submit(fetch_filings_for_ticker, ticker): ticker for ticker in tickers}
    for future in concurrent.futures.as_completed(future_to_ticker):
        ticker = future_to_ticker[future]
        try:
            filings = future.result()
            if filings:
                all_filings.extend(filings)
                print(f"Added {len(filings)} filings for {ticker}")
        except Exception as e:
            print(f"Error occurred for {ticker}: {e}")


print(f"Fetched data for {len(all_filings)} tickers")
all_filings_df = pd.DataFrame(all_filings) 
print(f"\nTotal filings collected: {len(all_filings_df)}")
if not all_filings_df.empty:
    print("\nDataFrame columns:", all_filings_df.columns.tolist())
    print("\nFirst few rows:")
    print(all_filings_df.head())
else:
    print("\nNo filings were collected")

Sample CIK mappings:
A: 1090872.0
AA: 1675149.0
AACB: 2034334.0
AACG: 1420529.0
AACT: 1853138.0
Added 5 filings for AADI
Added 1 filings for AAM
Added 2 filings for AACT
Added 6 filings for AAMI
Added 5 filings for AAME
Added 5 filings for A
Added 6 filings for AAL
Added 6 filings for AA
Fetched data for 36 tickers

Total filings collected: 36

DataFrame columns: ['ticker', 'formType', 'periodOfReport', 'filedAt', 'linkToFilingDetails', 'cik', 'accessionNumber']

First few rows:
  ticker formType periodOfReport                    filedAt  \
0   AADI     10-K     2023-12-31  2024-03-13T16:19:48-04:00   
1   AADI     10-K     2022-12-31  2023-03-28T17:34:21-04:00   
2   AADI     10-K     2021-12-31  2022-03-17T08:02:50-04:00   
3   AADI     10-K     2020-12-31  2021-03-11T08:24:15-05:00   
4   AADI     10-K     2019-12-31  2020-03-16T16:31:43-04:00   

                                 linkToFilingDetails         cik  \
0  https://www.sec.gov/Archives/edgar/data/142214...  01422142.0   
1

In [28]:
urls = all_filings_df[['ticker', 'formType', 'periodOfReport', 'filedAt', 'linkToFilingDetails']].rename(
    columns={'linkToFilingDetails': 'filingUrl'}
)
urls['financialReportsUrl'] = urls['filingUrl'].apply(lambda url: url.replace("ix?doc=", "").split("?")[0] + "/Financial_Report.xlsx")

print("\nURLs DataFrame info:")
print(f"Number of URLs: {len(urls)}")
print("\nFirst few rows:")
print(urls.head())


URLs DataFrame info:
Number of URLs: 36

First few rows:
  ticker formType periodOfReport                    filedAt  \
0   AADI     10-K     2023-12-31  2024-03-13T16:19:48-04:00   
1   AADI     10-K     2022-12-31  2023-03-28T17:34:21-04:00   
2   AADI     10-K     2021-12-31  2022-03-17T08:02:50-04:00   
3   AADI     10-K     2020-12-31  2021-03-11T08:24:15-05:00   
4   AADI     10-K     2019-12-31  2020-03-16T16:31:43-04:00   

                                           filingUrl  \
0  https://www.sec.gov/Archives/edgar/data/142214...   
1  https://www.sec.gov/Archives/edgar/data/142214...   
2  https://www.sec.gov/Archives/edgar/data/142214...   
3  https://www.sec.gov/Archives/edgar/data/142214...   
4  https://www.sec.gov/Archives/edgar/data/142214...   

                                 financialReportsUrl  
0  https://www.sec.gov/Archives/edgar/data/142214...  
1  https://www.sec.gov/Archives/edgar/data/142214...  
2  https://www.sec.gov/Archives/edgar/data/142214...  
3  htt

In [29]:
# urls = all_filings_df[['ticker', 'formType', 'periodOfReport', 'filedAt', 'linkToFilingDetails']].rename(
#     columns={'linkToFilingDetails': 'filingUrl'}
# )
# urls['financialReportsUrl'] = urls['filingUrl'].apply(lambda url: url.replace("ix?doc=", "").split("?")[0] + "/Financial_Report.xlsx")
# urls.head(10)
# print(all_filings_df.head())
# print(all_filings_df.columns)


In [30]:
#Download Financial_reports.xlsx of All EDGAR Filings
import os, requests

folder_path = './files'
API_KEY = '6d4bd60e237eae6df24af2c0189a10278154d4f2079dec475b7d15ca11baea04'
if not os.path.exists(folder_path):
  os.makedirs(folder_path)


def download_report(filing,api_key):
    try:
      reports_path = filing['financialReportsUrl'].replace('https://www.sec.gov/Archives/edgar/data/', '')
      base_url = f'https://archive.sec-api.io/{reports_path}'
      render_api_url = f'{base_url}?token={api_key}'

      response = requests.get(render_api_url)
      response.raise_for_status()
      print(f"Requesting: {render_api_url}")
      print(f"Response Code: {response.status_code}, Response Text: {response.text}")


      file_name = f"{filing['ticker']}-{filing['periodOfReport']}-{filing['formType']}.xlsx"
      file_path = f"{folder_path}/{file_name}"

      with open(file_path, 'wb') as output:
              output.write(response.content)
    except Exception as e:
          print(f"Error downloading report for {filing['ticker']}: {e}")
          

In [31]:
#!pip install -q pandarallel

# from pandarallel import pandarallel
# number_of_workers = 4
# pandarallel.initialize(progress_bar=False, nb_workers=number_of_workers, verbose=0)

In [ ]:
import concurrent.futures
import time
import requests

def construct_edgar_url(cik, accession_number, file_name):
    base_url = "https://www.sec.gov/Archives/edgar/data"
    return f"{base_url}/{cik}/{accession_number}/{file_name}"


def download_with_rate_limit(filing, api_key, retries=3, delay=5):
    folder_path = './files'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
    if not filing['cik'] or not filing['accessionNumber']:
        print(f"Missing CIK or accession number for {filing['ticker']}")
        return
        
    for attempt in range(retries):
        try:
            cik = str(filing['cik']).split('.')[0].zfill(10)  # Remove decimal and pad with zeros
            accession = filing['accessionNumber']

            sec_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{accession}/Financial_Report.xlsx"
            api_url = f"https://archive.sec-api.io/{cik}/{accession}/Financial_Report.xlsx?token={api_key}"

            current_url = sec_url if attempt == 0 else api_url
            print(f"\nAttempting to download from: {api_url}")

            headers = {
                'User-Agent': 'Mozilla/5.0',
                'Accept': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
            }
            
            response = requests.get(api_url, headers=headers)
            
            if response.status_code == 200:
                file_date = filing['filedAt'].split('T')[0]
                filename = f"{filing['ticker']}_{file_date}_Financial_Report.xlsx"
                file_path = os.path.join(folder_path, filename)
                
                content_type = response.headers.get('content-type', '')
                if 'application/json' in content_type.lower():
                    print(f"Received JSON instead of Excel file for {filing['ticker']}")
                    print(f"Response: {response.text[:200]}...")
                    continue
                
                with open(file_path, "wb") as f:
                    f.write(response.content)
                print(f"Successfully downloaded report for {filing['ticker']} to {file_path}")
                time.sleep(1)  # Rate limiting
                return True
            else:
                print(f"Failed to download report for {filing['ticker']}: Status {response.status_code}")
                if response.text:
                    print(f"Response: {response.text[:200]}...")

            if attempt < retries - 1:
                print(f"Retrying after {delay} seconds...")
                time.sleep(delay)
        except Exception as e:
            print(f"Error downloading {filing['ticker']}: {str(e)}")
            if attempt < retries - 1:
                delay_time = delay * (attempt + 1)  # Exponential backoff
                print(f"Retrying after {delay_time} seconds...")
                time.sleep(delay_time)
    return False

print("Sample filing data:")
print(all_filings_df[['ticker', 'cik', 'accessionNumber', 'filedAt']].head())

number_of_workers = 1  
successful_downloads = 0
failed_downloads = []

with concurrent.futures.ThreadPoolExecutor(max_workers=number_of_workers) as executor:
    futures = {
        executor.submit(download_with_rate_limit, filing, API_KEY): filing['ticker'] 
        for _, filing in all_filings_df.iterrows()
    }
    
    for future in concurrent.futures.as_completed(futures):
        ticker = futures[future]
        try:
            if future.result():
                successful_downloads += 1
            print(f"\nProgress: {successful_downloads}/{len(all_filings_df)} files downloaded")
        except Exception as e:
            print(f"\nError occurred for {ticker}: {str(e)}")

print(f"\nDownload completed. Successfully downloaded {successful_downloads} out of {len(all_filings_df)} reports")
print(f"Check the './files' directory for downloaded reports")

if failed_downloads:
    print("\nFailed downloads:")
    for filing in failed_downloads:
        print(f"- {filing['ticker']} from {filing['filedAt'].split('T')[0]}")

Sample filing data:
  ticker         cik     accessionNumber                    filedAt
0   AADI  01422142.0  000162828024010901  2024-03-13T16:19:48-04:00
1   AADI  01422142.0  000162828023009583  2023-03-28T17:34:21-04:00
2   AADI  01422142.0  000156459022010721  2022-03-17T08:02:50-04:00
3   AADI  01422142.0  000156459021012252  2021-03-11T08:24:15-05:00
4   AADI  01422142.0  000156459020011174  2020-03-16T16:31:43-04:00

Attempting to download from: https://archive.sec-api.io/0001422142/000162828024010901/Financial_Report.xlsx?token=6d4bd60e237eae6df24af2c0189a10278154d4f2079dec475b7d15ca11baea04
Successfully downloaded report for AADI to ./files\AADI_2024-03-13_Financial_Report.xlsx

Progress: 1/36 files downloaded
Attempting to download from: https://archive.sec-api.io/0001422142/000162828023009583/Financial_Report.xlsx?token=6d4bd60e237eae6df24af2c0189a10278154d4f2079dec475b7d15ca11baea04

Successfully downloaded report for AADI to ./files\AADI_2023-03-28_Financial_Report.xlsx



In [ ]:
with open("sec_finance_data_excel.ipynb", "r", encoding="utf-8", errors="replace") as f:
    content = f.read()

with open("sec_finance_data_excel_fixed.ipynb", "w", encoding="utf-8") as f:
    f.write(content)

print("Fixed notebook saved as sec_finance_data_excel_fixed.ipynb")
